# Task 3: Quantum SVM

We have the following task description for **Task 3** from [Cohort 7 Screening Tasks](https://docs.google.com/document/d/1KBot_q-CQ7FSmAXK45PDHNu8VKedOEbh/edit):

Generate a Quantum Support Vector Machine (QSVM) using the [iris dataset](https://archive.ics.uci.edu/ml/datasets/iris) and try to propose a kernel from a parametric quantum circuit to classify the three classes (setosa, versicolor, virginica) using the one-vs-all format, the kernel only works as binary classification. Identify the proposal with the lowest number of qubits and depth to obtain higher accuracy. You can use the UU† format or using the [Swap-Test](https://en.wikipedia.org/wiki/Swap_test).

## 0. Dependencies and imports

In [2]:
# !pip install scikit-learn==1.1.3

In [1]:
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt


np.random.seed(230306)

/usr/local/Caskroom/miniconda/base/envs/qosf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Load the dataset

In [2]:
X, y = load_iris(return_X_y=True)

The iris dataset consists of 4 numeric features and the target class:

* sepal length (cm)
* sepal width (cm)
* petal length (cm)
* petal width (cm)
* **class** (one of `'setosa'=0`, `'versicolor'=1`, `'virginica'=2`)

In [4]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
y[:5]

array([0, 0, 0, 0, 0])

In [14]:
y_class_dict = {"setosa": 0, "versicolor": 1, "virginica": 2}

## 2. Preprocess the dataset

Before the training step, we scale both `X` and `y` so that it is accommodated better by the algorithms we use.

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
X_scaled[:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

We scale the labels to $[-1,  1]$ for better performance with the SVM algorithm.

In [41]:
y_scaled = y - 1.

For one-vs-all approach, we set the target class to 1 and the rest to 0.

In [ ]:
# def y_transform(y, target_class):
#     y_return = y.copy()
#     y_return[y_return != target_class] = 0
#     y_return[y_return != 0] = 1
#     return y_return

In [ ]:
# y_setosa = y_transform(y, y_class_dict["setosa"])
# assert y[y == y_class_dict["setosa"]].sum() == y_setosa.sum()

In [ ]:
# y_versicolor = y_transform(y, y_class_dict["versicolor"])
# assert y[y == y_class_dict["versicolor"]].sum() == y_versicolor.sum()

We now split the dataset.

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

In [43]:
X_train.shape

(112, 4)

## 3. Quantum kernel

In [28]:
n_qubits = X_train.shape[1]
n_qubits

4

In [29]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

In [32]:
projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

In [33]:
projector

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [36]:
@qml.qnode(dev_kernel)
def quantum_kernel(x1, x2):
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [55]:
test_circuit_input = (X_train[0], X_train[0])

In [56]:
quantum_kernel(*test_circuit_input)

tensor(1., requires_grad=True)

We inspect our initial circuit a bit:

In [58]:
print(qml.draw(quantum_kernel)(*test_circuit_input))

0: ─╭AngleEmbedding(M0)─╭AngleEmbedding(M0)†─┤ ╭<𝓗(M1)>
1: ─├AngleEmbedding(M0)─├AngleEmbedding(M0)†─┤ ├<𝓗(M1)>
2: ─├AngleEmbedding(M0)─├AngleEmbedding(M0)†─┤ ├<𝓗(M1)>
3: ─╰AngleEmbedding(M0)─╰AngleEmbedding(M0)†─┤ ╰<𝓗(M1)>


In [53]:
specs_func = qml.specs(quantum_kernel)

In [59]:
specs_func(*test_circuit_input)

{'gate_sizes': defaultdict(int, {4: 2}),
 'gate_types': defaultdict(int,
             {'AngleEmbedding': 1, 'Adjoint(AngleEmbedding)': 1}),
 'num_operations': 2,
 'num_observables': 1,
 'num_diagonalizing_gates': 1,
 'num_used_wires': 4,
 'depth': 2,
 'num_trainable_params': 0,
 'num_device_wires': 4,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'autograd',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

# 4. SVC

We use the quantum kernel above to create a custom kernel for the scikit-learn `SVC` algorithm.

In [40]:
def custom_kernel_matrix(X, Y):
    """Computes the value of `kernel_function(x, y)` for each pair of x \in X, y \in Y into a matrix"""
    
    return [[quantum_kernel(x, y) for y in Y] for x in X]

In [48]:
svc = SVC(kernel=custom_kernel_matrix, decision_function_shape="ovr")
svc.fit(X_train, y_train)

SVC(kernel=<function custom_kernel_matrix at 0x7fa8511d1c60>)

In [49]:
dev_kernel.num_executions

12545

In [50]:
predictions = svc.predict(X_test)
accuracy_score(predictions, y_test)

0.9473684210526315

In [51]:
dev_kernel.num_executions

16801

For our initial quantum kernel that uses angle embedding for the input **(depth=2, n_qubits=4)**, we get a test accuracy of **0.9474** (to four decimal places). The next question is can we do better?